In [3]:
using Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating project at `~/Desktop/Work/Research/LEGO-project/LEGO-3D_Dyn`


In [4]:
import ForwardDiff as FD
using LinearAlgebra
using Plots

include(joinpath(@__DIR__,"helpers/quaternions.jl"))

quat2rot (generic function with 1 method)

In [ ]:
function newton_euler_dynamics_with_gravity(
  ᴮstates::Vector, 
  ᴮforcing::Vector, 
  params_rb::NamedTuple)

  q, ᴮv, ᴮω = ᴮstates[4:7], ᴮstates[8:10], ᴮstates[11:13]
  ᴮF, ᴮτ = ᴮforcing[1:3], ᴮforcing[4:6]

  Q = quat2rot(q)
  m, J, g = params_rb.m, params_rb.J, params_rb.g
  ᴺg = [0, 0, -g]
  ᴮg = Q' * ᴺg

  ᴺṙ = Q * ᴮv
  q̇ = (1/2) * L(q) * H * ᴮω
  ᴮv̇ = (1/m) * ᴮF + ᴮg - hat(ᴮω) * ᴮv
  ᴮω = J  \ (ᴮτ - hat(ᴮω) * (J * ᴮω))

  return [ᴺṙ; q̇; ᴮv̇; ᴮω̇]
end

function implicit_euler_dynamics_integrate(states::Vector, forcing::Vector, params_rb::NamedTuple, dt::Float64; max_iter=100, tol=1e-6)
  next_states = states # initialize next_states guess with the current states
  for i in 1:max_iter
    next_states_residual = states + newton_euler_dynamics_with_gravity(next_states, forcing, params_rb) * dt
    if norm(next_states_residual) < tol
      return next_states
    end
    # newton root finding 
    # TODO: check this
    next_states = next_states - FD.jacobian(next_states -> newton_euler_dynamics_with_gravity(next_states, forcing, params_rb), next_states) \ next_states_residual
  end
  throw("Implicit Euler dynamics integration did not converge")
end